In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from google.colab import drive

drive.mount('/content/gdrive')

data = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/bigdata/remove_city_under_5000.csv")

# 날짜 정보를 활용하여 연도와 월을 하나의 날짜 형식으로 변환
data['date'] = pd.to_datetime(data[['year', 'month']].assign(day=1))

# 학습 데이터와 테스트 데이터 분리
train_data = data[data['date'] < '2023-01-01']
test_data = data[data['date'] >= '2023-01-01']

# 도시별로 데이터를 분리하고 피처와 타겟으로 나누기
cities = data['city'].unique()
models = {}
predictions = {}

# 각 도시별로 모델 학습 및 예측 수행
for city in cities:
  city_data = data[data['city'] == city]

  # 월별 데이터 생성 (2009년 1월부터 2022년 12월까지)
  all_months = pd.date_range(start='2009-01-01', end='2022-12-01', freq='MS')
  city_months = city_data['date'].unique()
  missing_months = np.setdiff1d(all_months, city_months)
  missing_data = pd.DataFrame({'date': missing_months, 'city': city, 'people': np.nan})
  city_data = pd.concat([city_data, missing_data])
  city_data = city_data.sort_values(by='date')

  # NaN 값을 선형 보간을 사용하여 채워넣음
  city_data['people'] = city_data['people'].interpolate()

  # NaN 값을 제거하고 피처 데이터와 타겟 데이터 설정
  city_data = city_data.dropna()  # NaN 값을 제거
  X = city_data[['date']]
  y = city_data['people']

  # Random Forest 모델 생성 및 학습
  model = RandomForestRegressor(n_estimators=100, random_state=42)
  model.fit(X, y)

  # 예측을 위한 날짜 생성 (2023년 1월부터 2023년 7월까지)
  forecast_dates = pd.date_range(start='2023-01-01', end='2023-07-01', freq='MS')
  forecast_X = pd.DataFrame({'date': forecast_dates})

  # 예측
  forecast = model.predict(forecast_X[['date']])
  predictions[city] = forecast

# 예측 결과와 실제 데이터 비교 및 정확도 계산
accuracies = []

for city, forecast in predictions.items():
    city_test_data = test_data[test_data['city'] == city]

    if city_test_data['people'].isnull().any():
        # 만약 실제 데이터에 NaN 값이 있는 경우 해당 도시는 평균 정확도 계산에서 제외
        continue

    # 예측 결과와 실제 데이터 비교를 위한 날짜 생성
    forecast_dates = pd.date_range(start='2023-01-01', end='2023-07-01', freq='MS')
    forecast_df = pd.DataFrame({'date': forecast_dates, 'people_x': forecast})
    merged_df = forecast_df.merge(city_test_data, on='date', how='left')

    # NaN 값을 예측 값으로 채워넣음
    merged_df['people'] = merged_df['people'].fillna(merged_df['people_x'])

    # 예측 오차 계산 (RMSE)
    rmse = np.sqrt(mean_squared_error(merged_df['people_x'], merged_df['people']))

    if rmse == 0:
      continue

    # 백분율로 변환하여 정확도 계산
    actual_mean = merged_df['people'].dropna().mean()
    accuracy_percentage = (1 - (rmse / actual_mean)) * 100
    accuracies.append((city, rmse, accuracy_percentage))

# NaN 값이 없는 도시들의 평균 정확도 계산
average_accuracy = np.mean([accuracy_percentage for _, _, accuracy_percentage in accuracies])

# 결과 출력
for city, rmse, accuracy in accuracies:
    print(f"City: {city}, RMSE: {rmse:.2f}, Accuracy: {accuracy:.2f}%")

print(f"Average Accuracy for Cities without NaN: {average_accuracy:.2f}%")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
City: 간사이, RMSE: 3982.19, Accuracy: 98.53%
City: 괌, RMSE: 5798.38, Accuracy: 89.04%
City: 광저우, RMSE: 1822.40, Accuracy: 86.62%
City: 나고야, RMSE: 1866.62, Accuracy: 95.29%
City: 나리타, RMSE: 7536.80, Accuracy: 97.10%
City: 나트랑캄란, RMSE: 4732.75, Accuracy: 93.78%
City: 난징, RMSE: 585.45, Accuracy: 94.26%
City: 뉴욕, RMSE: 1833.81, Accuracy: 96.67%
City: 다낭, RMSE: 3241.84, Accuracy: 97.78%
City: 다싱국제공항, RMSE: 1630.84, Accuracy: 81.65%
City: 달라스, RMSE: 748.97, Accuracy: 96.77%
City: 대련, RMSE: 1375.79, Accuracy: 88.79%
City: 대북, RMSE: 3341.49, Accuracy: 97.49%
City: 덴파사, RMSE: 404.12, Accuracy: 98.21%
City: 델리, RMSE: 191.04, Accuracy: 98.68%
City: 도하, RMSE: 624.66, Accuracy: 96.98%
City: 돈무앙, RMSE: 1297.77, Accuracy: 69.47%
City: 두바이, RMSE: 902.53, Accuracy: 97.33%
City: 디트로이트, RMSE: 479.76, Accuracy: 96.96%
City: 라스베가스, RMSE: 524.15, Accuracy: 93.62%
City: 랑군, RMSE: 5

In [ ]:
!pip install pystan
!pip install prophet

In [ ]:
!pip install pmdarima